# Evaluación de revisión #2 LFYA

In [1]:
!python --version

Python 3.8.10


## 0.1 Install libraries abd set them up

In [2]:
!pip install -e /home/ivanvladimir/projects/maquinas

Obtaining file:///home/ivanvladimir/projects/maquinas
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: maquinas
    Found existing installation: maquinas 0.1.5.22
    Uninstalling maquinas-0.1.5.22:
      Successfully uninstalled maquinas-0.1.5.22
  Running setup.py develop for maquinas


In [3]:

!pip install rich
!pip install --upgrade -q gspread
!pip install gspread-formatting
!pip install fuzzywuzzy[speedup]
!pip install gdown
!pip install pandas
!pip install authlib


     |████████████████████████████████| 238 kB 1.5 MB/s eta 0:00:01
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
  Using cached python_Levenshtein-0.20.9-py3-none-any.whl (9.4 kB)
  Using cached Levenshtein-0.20.9-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (174 kB)
  Using cached rapidfuzz-2.13.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
     |████████████████████████████████| 78 kB 1.8 MB/s eta 0:00:01
  Using cached pandas-1.5.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
     |████████████████████████████████| 17.3 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 214 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 11.3 MB/s eta 0:00:01


In [1]:
#from google.colab import drive
from rich.progress import track as progress_bar
from rich.jupyter import print
from rich.console import Console
console = Console(width=200)
import pandas as pd
import gspread
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from googleapiclient.discovery import build
from gspread_formatting import *
from google.auth import default
from fuzzywuzzy import fuzz, process
import re
import urllib.request
import gdown
from IPython.display import SVG

## 0.2 Reading list and asignation

In [2]:
url_info = "" #@param {"type":"string"}

lista={}
lista_={}
with open("/home/ivanvladimir/Dropbox/Cursos/lfya/2023i/lista.txt") as f:
  for i,line in enumerate(f):
    line=line.strip()
    lista[i]=line.lower()
    lista_[line.lower()]=i

assign="""0,0,5,6,10
1,3,4,6,8
2,0,5,6,10
3,3,4,6,9
4,2,5,6,10
5,2,4,6,10
6,0,5,6,9
7,2,4,6,9
8,3,4,6,9
9,2,5,6,7
10,0,5,6,9
11,3,4,6,8
12,2,4,6,10
13,2,4,6,8
14,0,5,6,10
15,1,4,6,8
16,1,4,6,10
17,2,4,6,9
18,1,4,6,7
19,0,5,6,7
20,1,4,6,10
21,2,4,6,8
22,1,4,6,10
23,2,4,6,7
24,0,4,6,9
25,3,4,6,10
26,1,5,6,8
27,0,5,6,8
28,3,5,6,10
29,0,4,6,9
30,2,5,6,10
31,1,5,6,9
32,3,5,6,8
33,3,4,6,9
34,2,4,6,9
35,0,5,6,7
36,2,5,6,10
37,0,4,6,9
38,0,5,6,8
39,3,5,6,7
40,1,5,6,10
41,0,4,6,10
42,1,5,6,9
""".splitlines()
assign=[l.split(',') for l in assign]
assign=[tuple(int(v) for v in vals[1:]) for vals in assign]

print("\n".join(lista_.keys()))

alan bautista razo
alejandra rosales piña
alexis alejandro sanchez rosas
andres castrejon jimenez
angel isaac sanchez estrada
carlos morales ortega
cesar yahir perez diaz
christian peña granillo
christopher omar parra grimaldi
cinthya velez grande
daniel aguilar maya
diego andres ramirez garcia
eduardo abdiel mancera hernandez
eduardo alfredo ceron maciel
gabriel alexander carmona garcia
gabriela stephania figueroa villamares
gamaliel rios lira
giovanni salvador chavez villanueva
heber adan pilares garcia
hugo yael juarez garcia
ian pang araizaga
ikechukwu akwueze daniel
ivette alejandra carrillo cervantes
jesica villalpando aguilar
jesus javier jacobo ruiz
jorge antonio lopez chong
jose gerardo flores saavedra
karla andrea velazquez martinez
liliana alejandra muñiz hernandez
luis fabian armenta chora
luis joaquin castro serrato
marco antonio sanchez hernandez
maria fernanda verano peralta
martin gonzalez rico
miguel venegas rocha
rocio garcia lemus
rocio sanchez garcia
rodolfo mario teran garcia
rodrigo gael guzman alburo
samanta ivonne estrada romero
saul garcia lopez
vanessa nava alberto
yaxca alexa quero bautista

## 0.3 Connect to google sheets

In [3]:
gc = gspread.oauth()

In [4]:
#@title Opening excel survey

url_info = "https://docs.google.com/spreadsheets/d/1tvwObZXCbz2UETcD42GujvZ7qXcpkJOJayoXEDYc5mQ/edit?usp=sharing" #@param {"type":"string"}
ss= gc.open_by_url(url_info)
sh = ss.get_worksheet(0)
data  = sh.get_all_values()

## 0.4 Check who did not delivered

In [5]:
# Print columns
print("\n".join([f"{i:02} {col}" for i,col in enumerate(data[0])]))

00 Marca temporal
01 Dirección de correo electrónico
02 Puntuación
03 Nombre completo
04 Nombres trabajo en equipo
05 Anotar cinco cadenas que pertenezcan al lenguaje especificado
06 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
07 Transcribir los pasos del lemma de bombeo
08 Gramática libre de contexto
09 Argumente que la gramática es correcta
10 Nombres trabajo en equipo
11 Anotar cinco cadenas que pertenezcan al lenguaje especificado
12 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
13 Anotar el Autómata de Pila
14 Nombres trabajo en equipo
15 Resumen notación Bacus-Naur 
16 Segmento de if
17 Explique funcionamiento de segmento if
18 Nombres trabajo en equipo
19 Anotar cinco cadenas que pertenezcan al lenguaje especificado
20 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
21 Anotar el Autómata de Pila

In [6]:
errs=0
id2rows={}
for i,row in enumerate(data[1:]):
  res=process.extract(row[3].lower(),lista_.keys(),limit=1)
  if len(res)==0:
    print(f"[red]No match found: {row[3].lower()} [/red]")
    errs+=1
  else:
    res=res[0]
    if res[1]<85:
      print(f"[red]No enough evidence: {row[3].lower()} [/red]")
      errs+=1
    else:
      id2rows[lista_[res[0]]]=row

if errs > 0:
  print("[red]❌ it did not pass[/red]")
else:
  print(f"[green]✓ it seems to be OK, total found {len(id2rows)}[/green]")

✓ it seems to be OK, total found 42

## 0.5 Functions

In [17]:
from maquinas.io import *
from maquinas.regular.ndfa_e import NonDeterministicFiniteAutomaton_epsilon as NDFA_e
from maquinas.regular.ndfa import NonDeterministicFiniteAutomaton as NDFA
from maquinas.regular.dfa import DeterministicFiniteAutomaton as DFA

from maquinas.regular.RE import RegularExpression as RE
from maquinas.contextfree.cfg import ContextFreeGrammar as CFG
from maquinas.contextfree.pda import PushDownAutomaton as PDA
from maquinas.recursivelyenumerable.tspda import TwoStackPushDownAutomaton as TSPDA

PDA2=TSPDA


from maquinas.parser.earley_parser import print_chart, chart2table


class Responses():

  def __init__(self):
    self.res={}
    self.num={}

  def set_problem(self,num):
    self.num=num
    if not num in self.res:
      self.res[self.num]={}

  def append(self,type,res):
    if not type in self.res[self.num]:
      self.res[self.num][type]=[]
    self.res[self.num][type].append(res)

  def concat(self,type,res):
    if not type in self.res[self.num]:
      self.res[self.num][type]=[]
    self.res[self.num][type]+=res

  def get(self,num,type):
    return self.res[num][type]

def clean_accept_reject(w):
  w=w.strip()
  if w.startswith('w='):
    w=w[2:]
  w=w.strip()
  w=w.replace("-","")
  w=w.replace(",","")
  w=w.replace(";","")
  w=w.replace(".","")
  if w=="Ɛ":
        w=''
  
  return w

re_http=re.compile("(https?://.*)")
re_docs_id=re.compile(r"/d/([^/]+)/")

def find_url(text):
  m=re_http.search(text)
  if m:
    return m.group(1)
  else:
    return None


def load_code(text,type):
  trans=[]
  
  for line in text.splitlines():
    
    line=str(line.strip())
    if len(line)==0:
        continue
    if "States" in line:      
      states=line[10:].split(',')
      states=[a.strip() for a in states]
    if line.startswith("Sigma  "):
      line=line.replace("ε","")
      sigma=line[10:].split(',')
      sigma=[s.strip() for s in sigma]
    if line.startswith("Initial "):
      q0=line[10:].strip()
    if "Aceptors" in line:
      A=line[11:-1].split(',')
      A=[a.strip() for a in A]
    if "Acceptors" in line:
      A=line[12:-1].split(',')
      A=[a.strip() for a in A]
    
    if "→" in line:
      
      bits=line.split("→")
      bits0=bits[0].split(',')
      if type=="NDFA_e":
        if not bits[1].strip()[0]=='{':
          print("[red]It did not found a set in transition of NDFA_e[/red]")
          return None
      if len(bits[1].strip())==2:
        continue
    
      if type=="DFA":
        trans.append(((bits0[0].strip(),bits0[1].strip()),bits[1].strip()))
      else:
          bits1=bits[1].strip()[1:-1].split(',')
          trans.append(((bits0[0].strip(),bits0[1].strip()),[b.strip() for b in bits1]))
      
  if type=="NDFA_e":
    try:
      return NDFA_e(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type=="NDFA":
    try:
      return NDFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type=="DFA":
    try:
      return DFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type==None:
    
     return NDFA_e(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
     return NDFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
     return DFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    
    
    
re_transition=re.compile(r"  ([^,\n]+,[^ ]+) →")
re_epsilon=re.compile(r",e →")

def create_pda(text):
  if "<?xml" in text:
    #text=text.replace("&#13;","")
    text="<structure>"+text.split("<structure>")[1]
    text=text.split("</structure>")[0]+"</structure>\n"
    try:
        return load_jflap(text)
    except JFLAPXMLError:
        return None,None
  elif "RE(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("RE(")
    try:
        return eval("RE("+bits[1])
    except:
        print(text)
  elif "PDA(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("PDA(")
    PDA_="PDA("+bits[1]
    PDA_=PDA_.split("bvmbv.graph()")[0]
    PDA_=PDA_.split("anbnMio")[0]
    PDA_=PDA_.split("for q,a,w_")[0]
    
    try:
        return eval(PDA_)
    except:
        print(text)
  elif "|" in text.split('\n')[0]:
      return load_fa(text)

def create_pda2(text):
  if "<?xml" in text:
    #text=text.replace("&#13;","")
    text="<structure>"+text.split("<structure>")[1]
    text=text.split("</structure>")[0]+"</structure>\n"
    try:
        return load_jflap(text)
    except JFLAPXMLError:
        return None,None
  elif "States" in text:
    text=text.replace("Sigma","\nSigma")
    text=text.replace("Initial","\nInitial")
    text=text.replace("Acceptors","\nAcceptors")
    text=text.replace("Aceptors","\nAcceptors")
    text=text.replace("Transitions","\nTransitions")
    text=re_transition.sub(r"\n\1 →",text)
    text=re_epsilon.sub(r",ε →",text)
    return gid,load_code(text,type)
  elif "RE(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("RE(")
    try:
        return eval("RE("+bits[1])
    except:
        print(text)
  elif "PDA2(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("PDA2(")
    PDA2_="PDA2("+bits[1]
    PDA2_=PDA2_.split("E8_1.graph()")[0]
    PDA2_=PDA2_.split("wmw.graph()")[0]
    PDA2_=PDA2_.split("E9.graph()")[0]
    PDA2_=PDA2_.split("for q,a,w_ in")[0]
    PDA2_=PDA2_.split("w1mw2mw3mw1w1w2.graph()")[0]

    
    
        
    return eval(PDA2_)
  elif "|" in text.split('\n')[0]:
      return load_fa(text)


## Section 1
---


In [24]:
solutions=[
    # section 1
    {0: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['0001','01','0010','00010','00001'],
      "reject":['1','010','0100','10','001000'],
     },
     1: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['1110','01','101111','10111','01111'],
      "reject":['1','101','1101','10','111011'],
     },
     2: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['1000','01','0010','00010','00001'],
      "reject":['1','010','0100','10','001000'],
     },
     3: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['0111','01','1011','10111','01111'],
      "reject":['1','101','1101','10','111011'],
     },
     
    },
    {4: # problema dos
     {"alpabeth":set("a b m".split()),
      "accept":['baaabmb','aaabbmb','amb','bbbaaaaaabbmb','baabm'],
      "reject":['aaaa','bbb','am','mb','baaabmmb'],
     },
     5: # problema dos
     {"alpabeth":set(['a','b','m']),
      "accept":['bbbmbbb',"bbmbb","bmb","bbmbb","m"],
      "reject":['bmbb','bbb','bbmb','bbmbbb','bbbmbb'],
      
     }},
    {
     7: # problema cuatro
     {"alpabeth":set(['a','b','m']),
      "accept":['ambmambaa','aamaambbmaabbaa','mmm','bmambmabb','mmama'],
      "reject":['a','m','mm','ambmamaaa','amamamaa'],      
     },
        8: # problema cuatro
     {"alpabeth":set(['a','b','m']),
      "accept":['ambmamaab','aamaambbmbbaaaa','mmm','bmambmbba','mmama'],
      "reject":['a','m','mm','ambmamaaa','amamamaa'],      
     },
        9: # problema cuatro
     {"alpabeth":set(['a','b','m']),
      "accept":['ambmamaaa','aamaambbmaabbbb','mmm','bmambmbbb','mmamaa'],
      "reject":['a','m','mm','ambmamaaa','amamamaa'],      
     },
        10: # problema cuatro
     {"alpabeth":set(['a','b','m']),
      "accept":['ambmamaab','aamaambbmaaaaaa','mmm','bmambmbba','mmam'],
      "reject":['a','m','mm','ambmamaaa','amamamaa'],      
     },
    
    
    
    },
    
]    

In [30]:
responses=Responses()
doc2name={}
FB={}
POINTS={}

for num,(id,row) in list(enumerate(id2rows.items()))[40:]:
  responses.set_problem(assign[id][0])
  problem=assign[id][0]
  # recover name
  name=row[3].strip()
  POINTS[name]=[5,5,11,10,5,
                5,5,16,
                5,5,5,
                5,5,13]
  FB[name]=["" for _ in POINTS[name]]
  print(f"[yellow]{name}/{row[1]}/{num+1} ({problem}) [/yellow]")

  ### Seccion uno
  print("[magenta]SECTION 1[/magenta]",f"[blue]Problem {problem+1}[/]")
  accept=[clean_accept_reject(a) for a in row[5].split() if a not in ['•']]
  
  for a in accept:
      for a_ in a:
          if '}' in a:
            FB[name][0]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
          elif not a_ in solutions[0][problem]['alpabeth']:
            FB[name][0]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break
      a_=a
      
      if problem==0:
        bits=a_.split('1')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])>len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==1:
        bits=a_.split('0')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])<len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==2:
        bits=a_.split('1')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not ((len(bits[0])>len(bits[1])) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==3:
        bits=a_.split('0')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])<len(bits[1]) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
        
            
  reject=[clean_accept_reject(r) for r in row[6].split() if r not in ['•{}']]

  for a in reject:
      for a_ in a:
          if '}' in a:
            FB[name][1]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
          elif not a_ in solutions[0][problem]['alpabeth']:
            FB[name][1]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
            break
      a_=a
      
      if problem==0:
        bits=a_.split('1')
        if len(bits)!=2:
            break

        if (len(bits[0])>len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==1:
        bits=a_.split('0')
        if len(bits)!=2:
             break

        if (len(bits[0])<len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==2:
        bits=a_.split('1')
        if len(bits)!=2:
            break

        if ((len(bits[0])>len(bits[1])) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==3:
        bits=a_.split('0')
        if len(bits)!=2:
            break

        if (len(bits[0])<len(bits[1]) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
            
            
      if len(row[7].strip())==0:
            FB[name][2]+=f"No se definió parte de la cadena dependiente de n\n"
            POINTS[name][2]=max(0,POINTS[name][2]-1)
        
    
            FB[name][3]+=f"No se particionó la cadena\n"
            POINTS[name][3]=max(0,POINTS[name][3]-1)
            
            FB[name][4]+=f"No se aplicó el lema del bombeo\n"
            POINTS[name][4]=max(0,POINTS[name][4]-1)
            
      if len(row[9].strip())==0:
            FB[name][6]+=f"No se argumentó la gramática\n"
            POINTS[name][6]=max(0,POINTS[name][6]-1)
  ps=[]
  g_=re.sub(r'\{.*\}',"",row[8])
  for r in re.split('\n+', g_):
        r=r.replace('donde P:','')
        r=r.replace('Donde P:','')
        r=r.replace('P:','')
        r=r.replace('(, P,S)','')
        r=r.replace('G=','')
        
        if not len(r)>0:
            continue
        h,bs=r.strip().split('->',1)
        bs=bs.split('|')
        for b in bs:
            ps.append(f"{h.strip()} -> {b.strip()}")
  g="; ".join(ps)

  grammar_1=CFG(g)
    

  for a in solutions[0][problem]['accept']:
    if not grammar_1.accepts(a):
        FB[name][3]+=f"No se genera la cadena '{a}' en la gramática\n"
        POINTS[name][3]=max(0,POINTS[name][3]-1)

  for a in solutions[0][problem]['reject']:
    if grammar_1.accepts(a):
        FB[name][3]+=f"Se genera la cadena '{a}' en la gramática que no se debería \n"
        POINTS[name][3]=max(0,POINTS[name][3]-1)
   
  problem=assign[id][1] 
  print("[magenta]SECTION 2[/magenta]",f"[blue]Problem {problem+1}[/]")
  
  accept=[clean_accept_reject(a) for a in row[11].split() if a not in ['•']]
  for a in accept:
      for a_ in a:
          if '}' in a:
            FB[name][5]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][5]=max(0,POINTS[name][5]-1)
          elif not a_ in solutions[1][problem]['alpabeth']:
            FB[name][5]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][5]=max(0,POINTS[name][5]-1)
            break
      a_=a
      
      if problem==4:
        bits=a_.split('m')
        if len(bits)!=2:
            FB[name][5]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][5]=max(0,POINTS[name][5]-1)
            break

      if problem==5:
        bits=a_.split('m')
        if len(bits)!=2:
            FB[name][5]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][5]=max(0,POINTS[name][5]-1)
            break

  
  reject=[clean_accept_reject(r) for r in row[12].split() if r not in ['•{}']]

  for a in reject:
      for a_ in a:
          if '}' in a:
            FB[name][6]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][6]=max(0,POINTS[name][6]-1)
          elif not a_ in solutions[1][problem]['alpabeth']:
            FB[name][6]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][6]=max(0,POINTS[name][6]-1)
            break
      a_=a
      
      if problem==4:
        bits=a_.split('1')
        if len(bits)!=2:
            break

      if problem==5:
        bits=a_.split('0')
        if len(bits)!=2:
             break  

  ps=[]
  ap=row[13]
  ap=ap.replace("&#13;","")
  
  pda=create_pda(ap)

  for a in solutions[1][problem]['accept']:
    if not pda.accepts(a):
        FB[name][7]+=f"No se acepta a la cadena '{a}' en el pda\n"
        POINTS[name][7]=max(0,POINTS[name][7]-1)

  for a in solutions[1][problem]['reject']:
    if pda.accepts(a):
        FB[name][7]+=f"Se acepta la cadena '{a}' en el pda que no se debería \n"
        POINTS[name][7]=max(0,POINTS[name][7]-1)
  
  problem=assign[id][3]
  print("[magenta]SECTION 4[/magenta]",f"[blue]Problem {problem+1}[/]")
  
  accept=[clean_accept_reject(a) for a in row[19].split() if a not in ['•']]
  for a in accept:
      for a_ in a:
          if '}' in a:
            FB[name][11]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
          elif not a_ in solutions[2][problem]['alpabeth']:
            FB[name][11]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
            break
      a_=a
      
      if problem==4:
        bits=a_.split('m')
        if len(bits)!=4:
            FB[name][11]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
            break

        if not (len(bits[0])>len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][11]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
      if problem==5:
        bits=a_.split('m')
        if len(bits)!=4:
            FB[name][11]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
            break

        if not (len(bits[0])<len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][11]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][11]=max(0,POINTS[name][11]-1)
  
  reject=[clean_accept_reject(r) for r in row[20].split() if r not in ['•{}']]

  for a in reject:
      for a_ in a:
          if '}' in a:
            FB[name][12]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][12]=max(0,POINTS[name][12]-1)
          elif not a_ in solutions[2][problem]['alpabeth']:
            FB[name][12]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][12]=max(0,POINTS[name][12]-1)
            break
      a_=a
      

  ps=[]
  ap=row[21].strip()
  ap=ap.replace("&#13;","")
  
  if len(ap)>0:
      pda=create_pda2(ap)
  else:
      pda=PDA2()

  for a in solutions[2][problem]['accept']:
    if not pda.accepts(a):
        FB[name][13]+=f"No se acepta a la cadena '{a}' en el doble pda\n"
        POINTS[name][13]=max(0,POINTS[name][13]-1)

  for a in solutions[2][problem]['reject']:
    if pda.accepts(a):
        FB[name][13]+=f"Se acepta la cadena '{a}' en el doble pda que no se debería \n"
        POINTS[name][13]=max(0,POINTS[name][13]-1)


  ### PRINTS RESULTS
  total=0
  for i,(fb,p) in enumerate(zip(FB[name],POINTS[name])):
        print(f"{i}: {p}",fb)
        total+=p
  print("Total resultados:", total)

Christopher Omar Parra Grimaldi/subarusrt24@gmail.com/41 (3) 

SECTION 1 Problem 4

SECTION 2 Problem 5

SECTION 4 Problem 10

0: 5

...

10: 5

11: 5

12: 5

13: 13

Total resultados: 100